In [162]:
from selenium import webdriver
import pandas as pd
import re
import time
import threading
import queue
import numpy as np

# 设置浏览器

In [163]:
url = 'http://tcmssd.ratcm.cn/#/Herb?id=1'

browser_0 = webdriver.Chrome()
browser_0.get('http://tcmssd.ratcm.cn/#/Herb?id=1')
time.sleep(3)

browser_1 = webdriver.Chrome()
browser_1.get('http://tcmssd.ratcm.cn/#/Herb?id=1')
time.sleep(3)

browser_2 = webdriver.Chrome()
browser_2.get('http://tcmssd.ratcm.cn/#/Herb?id=1')
time.sleep(3)

# 三步走计划

In [164]:
def step1_visit_website(base_url, TCM_index, browser_index):
    """
    访问网站链接并获取网页源码
    :param browser_index:
    :param TCM_index:数据库中的方剂编号
    :param base_url:网址基础链接
    :return: 网页源代码的str格式
    """
    #browser = webdriver.Chrome()

    if browser_index == 0:
        browser_0.refresh()
        target_url = base_url + str(TCM_index)
        browser_0.get(target_url)
        browser_0.refresh()
        time.sleep(3)
        source_code = browser_0.page_source
        step2_wash_data(source_code, TCM_index)
        browser_0.refresh()
    elif browser_index == 1:
        browser_1.refresh()
        target_url = base_url + str(TCM_index)
        browser_1.get(target_url)
        browser_1.refresh()
        time.sleep(3)
        source_code = browser_1.page_source
        step2_wash_data(source_code, TCM_index)
        browser_1.refresh()
    elif browser_index == 2:
        browser_2.refresh()
        target_url = base_url + str(TCM_index)
        browser_2.get(target_url)
        browser_2.refresh()
        time.sleep(3)
        source_code = browser_2.page_source
        step2_wash_data(source_code, TCM_index)
        browser_2.refresh()

    return 0


#url = 'http://tcmssd.ratcm.cn/#/Prescription?id='
#step1_visit_website(url, 1)

In [165]:
def step2_wash_data(data, TCM_index):
    """
    清洗源码数据以获得方剂信息
    :param TCM_index:
    :param data:源码
    :return:方剂信息
    """

    Name_Form_pattern = r">中草药名称（中文）<.*?><.*?>(.*?)<\/td>"
    Name_Form_matches = re.findall(Name_Form_pattern, data)

    SQWW_Form_pattern = r">四气五味<.*?><.*?>(.*?)<\/td>"
    SQWW_Form_matches = re.findall(SQWW_Form_pattern, data)

    Guijing_Form_pattern = r">中药归经<.*?><.*?>(.*?)<\/td>"
    Guijing_Form_matches = re.findall(Guijing_Form_pattern, data)

    Source_Form_pattern = r"<a data-v-4a1273a5="".*?>(.*?)<\/a>"
    Source_Form_matches = re.findall(Source_Form_pattern, data)

    Function_Form_pattern = r">功效主治<.*?><.*?>(.*?)<\/td>"
    Function_Form_matches = re.findall(Function_Form_pattern, data)

    Usage_Form_pattern = r">用法用量<.*?><.*?>(.*?)<\/td>"
    Usage_Form_matches = re.findall(Usage_Form_pattern, data)

    Formula_Form_pattern = r">方剂<.*?><.*?>(.*?)<\/td>"
    Formula_Form_matches = re.findall(Formula_Form_pattern, data)

    #print((Name_Form_matches, SQWW_Form_matches, Guijing_Form_matches, Source_Form_matches, Function_Form_matches,Usage_Form_matches, Formula_Form_matches, TCM_index))

    step3_push_excel(Name_Form_matches, SQWW_Form_matches, Guijing_Form_matches, Source_Form_matches, Function_Form_matches,Usage_Form_matches, Formula_Form_matches, TCM_index)

    return 0

#url = 'http://tcmssd.ratcm.cn/#/Prescription?id='
#data = step1_visit_website(url, 1)
#step2_wash_data(data)

In [166]:
def step3_push_excel(Name, SQWW, Guijing, Source, Function, Usage, Formula, row_index):
    """

    :return:
    """

    global HERB_Chem


    TCMSSD_HERB.loc[row_index, '名称'] = Name[0]
    TCMSSD_HERB.loc[row_index, '四气五味'] = SQWW[0]
    TCMSSD_HERB.loc[row_index, '中药归经'] = Guijing[0]
    TCMSSD_HERB.loc[row_index, '方剂'] = Formula[0]
    TCMSSD_HERB.loc[row_index, '功效主治'] = Function[0]
    TCMSSD_HERB.loc[row_index, '用法用量'] = Usage[0]

    if len(Source) > 0:
        new_rows = pd.DataFrame({'中药': [Name[0]] * len(Source), '成分': Source})
        HERB_Chem = pd.concat([HERB_Chem, new_rows], ignore_index=True)

    print(row_index)

    return 0


# 主程序

In [167]:
global TCMSSD_HERB
global HERB_Chem

TCMSSD_HERB = pd.read_excel("D:/github_planting/MGM/pre_data/TCMSSD_HERB.xlsx", sheet_name=0)

HERB_Chem = pd.read_excel("D:/github_planting/MGM/pre_data/TCMSSD_HERB_Chem.xlsx", sheet_name=0)

In [168]:
TCMSSD_HERB

,名称,四气五味,中药归经,成分,功效主治,用法用量,方剂


In [169]:
HERB_Chem

,中药,成分
0,牡荆根,aucubigenin
1,纤细薯蓣,dioscoreside c
2,纤细薯蓣,dioscin
3,纤细薯蓣,protodioscin
4,纤细薯蓣,yamogenin acetate
5,纤细薯蓣,yamogenin
6,纤细薯蓣,protogracillin(ii)
7,纤细薯蓣,diosgenin
8,纤细薯蓣,protogracillin
9,纤细薯蓣,Δ3


In [170]:
formula_ID = np.arange(1, 8257)
formula_ID_queue = queue.Queue()

for for_id in formula_ID:
    formula_ID_queue.put(for_id)

In [171]:
def crawl(browser_index):
    while not formula_ID_queue.empty():
        ID = formula_ID_queue.get()
        step1_visit_website(url, ID, browser_index)
        formula_ID_queue.task_done()

In [172]:
thread_list = []

start_time = time.time()

for i in range(3):
    thread_list.append(threading.Thread(target=crawl, args=(i, )))

for t in thread_list:
    t.start()
for t in thread_list:
    t.join()

2
3
1
54

6
7
9
8
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
53
52
54
55
56
57
58
59
60
61
63
62
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
186185

187
188
189
190
191
192
193
194
195
196
198
197
199
200
201
202
204203

205
207206

208
209
210
211
212
213
214
215
216
217
218
220
219
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
244
243
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
262
261
263
265
264
266
267
268
269
270
271
272
273
274
275
276
277


In [176]:
TCMSSD_HERB.to_excel("D:/github_planting/MGM/pre_data/TCMSSD_HERB.xlsx", index=False)
HERB_Chem.to_excel("D:/github_planting/MGM/pre_data/TCMSSD_HERB_Chem.xlsx", index=False)

In [ ]:
end_time= time.time()
print(str(end_time - start_time))

In [179]:
TCMSSD_HERB

,名称,四气五味,中药归经,成分,功效主治,用法用量,方剂
3,凹叶瑞香,,,NaN,See Daphne tangutica.,See Daphne tangutica.,"<span data-v-4a1273a5=""""></span>"
1,牡荆根,Warm;Pungent; Slight sweet,Lung;Liver;Spleen,NaN,,,"<span data-v-4a1273a5=""""></span>"
5,,,,NaN,,,"<span data-v-4a1273a5=""""></span>"
4,云南含笑,,,NaN,To clear heat and resolve toxin.,"Pharyngolaryngitis, nasitis, conjunctivitis, b...","<span data-v-4a1273a5=""""></span>"
6,夏至草,,,NaN,"To nourish body and quicken blood, clear heat ...","Menstrual disorder, postpartum abdominal pain ...","<span data-v-4a1273a5=""""></span>"
...,...,...,...,...,...,...,...
8252,,,,NaN,,,"<span data-v-4a1273a5=""""></span>"
8253,,,,NaN,,,"<span data-v-4a1273a5=""""></span>"
8254,,,,NaN,,,"<span data-v-4a1273a5=""""></span>"
8255,,,,NaN,,,"<span data-v-4a1273a5=""""></span>"


In [180]:
TCMSSD_HERB['名称'].isnull().sum()

77

In [191]:
def NLP(content):
    temp_pattern = r"<.*?>(.*?)</span>"
    temp_matches = re.findall(temp_pattern, content)
    print(temp_matches)
    return temp_matches

In [192]:
TCMSSD_HERB = pd.read_excel("D:/github_planting/MGM/pre_data/TCMSSD_HERB.xlsx", sheet_name=0)

In [193]:
TCMSSD_HERB['方剂'] = TCMSSD_HERB['方剂'].apply(NLP)

['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['护岩膏']
['烧丹散,榄核散,喉癣吹药,三神散,冰六散,黑香散,华佗累效散,胡桃饮,敛肌散,橄榄散']
['']
['']
['']
['']
['']
['苦参酒,扶正消毒饮,花叶洗剂,五味消毒化疔饮,鼻窦灌注液,松花膏,洗疥药,臭梧桐洗剂,湿疹合剂,还睛散,降痈散,苦参水,拔疔膏,明目紫金膏,扶危散,野菊煎剂,柴胡消痈汤,凉血五花汤,东篱散,消炎合剂,瘰疬酒药,解急饮,清热解毒消肿汤,五味消毒饮']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['一味秘精汤']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['紫苏汤,桂枝加厚朴杏子汤,泻肝汤,桂枝加桂汤,桂枝瓜蒌首乌汤,柴芩栝楼芍药汤,亥骨饮,柴桔汤,芒消汤,白茯苓汤,杏仁粥,补肺汤,加味清宁膏,食柏圣饼子,当归补血汤,桂枝加葛根汤,麻黄连轺赤小豆汤,牵牛子丸,人参汤,薤白汤,乌龙丸,温中丸,紫苏石膏地黄汤,秘传加味四君子汤,蓖麻丸,安中汤,托里养荣汤,木通桂枝汤,文蛤汤,升阳益气汤,何首乌丸,桂枝附子汤,填骨万金煎,镇心丸,小柴胡去参枣生姜加五味子干姜汤,三白汤,大风引汤,解肌汤,清心汤,黄连汤,胶芩四君汤,越婢加术汤,黄耆汤,败弩筋散,金蟾散,五辛汤,桑根白皮汤,神妙绝疟汤,泻肺汤,姜枣丸,厚朴散,小定心汤,黄连煎,当归汤,草灵丹,补肝汤,轻粉散,葛根解托汤,大鳖甲汤,断痢汤,锦灰散,泽兰子汤,封脐雄鼠粪散,胆矾丸,小柴胡加牡丹皮汤,半夏汤,和胃汤,半夏桂枝汤,黑香四神散,艾胶汤,加味六君汤,二加龙骨汤,桃仁汤,柴胡加龙骨牡蛎汤,加味安胎饮,竹茹汤,茱萸泽泻乌头桂枝汤,治要茯苓补心汤,菊花汤,乌金丸,大黄汤,大枣膏,加减养心汤,葛根解肌汤,银白丹,加味黄芩汤,芍药黄耆汤,点眼金丝膏,保真汤,小葶苈汤,异功快斑汤,六般芍药饮,补肺散,朱雀汤,防风煮散,黄耆鳖甲饮,托里温中汤,桂枝加当归茯苓汤,黄耆当归散,调中汤,草果饮子,分气散,葶苈五子汤,青皮饮,弩筋散,臭橙饮,葛根汤,救命汤,人参应梦散,桂枝加耆术汤,滋阴补心汤,凉膈散,吴茱萸汤,建脾散,和阴益阳汤,

In [194]:
TCMSSD_HERB

,名称,四气五味,中药归经,成分,功效主治,用法用量,方剂
0,凹叶瑞香,NaN,NaN,NaN,See Daphne tangutica.,See Daphne tangutica.,[]
1,牡荆根,Warm;Pungent; Slight sweet,Lung;Liver;Spleen,NaN,NaN,NaN,[]
2,NaN,NaN,NaN,NaN,NaN,NaN,[]
3,云南含笑,NaN,NaN,NaN,To clear heat and resolve toxin.,"Pharyngolaryngitis, nasitis, conjunctivitis, b...",[]
4,夏至草,NaN,NaN,NaN,"To nourish body and quicken blood, clear heat ...","Menstrual disorder, postpartum abdominal pain ...",[]
...,...,...,...,...,...,...,...
8223,NaN,NaN,NaN,NaN,NaN,NaN,[]
8224,NaN,NaN,NaN,NaN,NaN,NaN,[]
8225,NaN,NaN,NaN,NaN,NaN,NaN,[]
8226,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [196]:
TCMSSD_HERB.to_excel("D:/github_planting/MGM/pre_data/TCMSSD_HERB.xlsx", index=False)